In [ ]:
!pip install --user boto3
!pip install --user minio
!pip install --user mlxtend
!pip install --user pandas


In [1]:
import boto3
import io
import pandas as pd
import os
from minio import Minio
from minio.error import S3Error

In [2]:
minio_credential = {
    'region': 'local',
    'endpoint':'http://127.0.0.1:8000',
    'use_ssl': False,
    'url_style':'path',
    'access_key':'admin',
    'secret_key':'password'
}

s3 = boto3.client(
    's3',
    region_name=minio_credential['region'],
    endpoint_url=minio_credential['endpoint'],
    use_ssl=minio_credential['use_ssl'],
    aws_access_key_id=minio_credential['access_key'],
    aws_secret_access_key=minio_credential['secret_key'],
    config=boto3.session.Config(signature_version='s3v4')
)

In [3]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '179F2330A65B1B9C',
  'HostId': '03f6d7ba09b0531a178059659f12e65ab6a75adddf2f548b1f37624d55d95fba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '639',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': '03f6d7ba09b0531a178059659f12e65ab6a75adddf2f548b1f37624d55d95fba',
   'x-amz-request-id': '179F2330A65B1B9C',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 09 Dec 2023 10:31:16 GMT'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'bucket1',
   'CreationDate': datetime.datetime(2023, 11, 6, 4, 27, 44, 555000, tzinfo=tzutc())},
  {'Name': 'csv',
   'CreationDate': datetime.datetime(2023, 12, 7, 13, 0, 49, 286000, tzinfo=tzutc())},
  {'Name': 'parquet',
   'CreationDate': datetime.datetime(2023, 12, 7, 13, 2, 52, 405000, tzinfo=

In [4]:
from mlxtend.frequent_patterns import apriori, association_rules

In [5]:
minio_client = Minio(
    "127.0.0.1:8000",
    access_key="admin",
    secret_key="password",
    secure=False  
)


data_object = minio_client.get_object('csv', 'e commerce/Online_Retail.csv')

In [6]:
df = pd.read_csv(data_object)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [17]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [8]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [7]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [16]:
transaction_France = (df[df['Country']=="France"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
transaction_Netherlands = (df[df['Country']=="Netherlands"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

In [19]:
def hot_encode(x):
    if (x<=0):
        return 0
    if (x>=1):
        return 1
    

France_encoded = transaction_France.applymap(hot_encode)
Netherlands_encoded = transaction_Netherlands.applymap(hot_encode)

C:\Users\andar\AppData\Local\Temp\ipykernel_14544\332089142.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  France_encoded = transaction_France.applymap(hot_encode)
C:\Users\andar\AppData\Local\Temp\ipykernel_14544\332089142.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Netherlands_encoded = transaction_Netherlands.applymap(hot_encode)


In [33]:
model = apriori(France_encoded, min_support=0.05, use_colnames=True)
rules = association_rules(model, metric="lift", min_threshold=1)
rules = rules.sort_values(['confidence','lift'], ascending=[False,False])
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
44,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf,0.254144
259,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf,0.247312
271,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf,0.247978
301,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959,0.967949
300,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796,0.956294


In [22]:
model = apriori(Netherlands_encoded, min_support=0.05, use_colnames=True)
rules = association_rules(model, metric="lift", min_threshold=1)
rules = rules.sort_values(['confidence','lift'], ascending=[False,False])
rules.head()

C:\Users\andar\AppData\Roaming\Python\Python310\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
650,(FOLDING BUTTERFLY MIRROR HOT PINK),(FOLDING BUTTERFLY MIRROR RED),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf,1.0
651,(FOLDING BUTTERFLY MIRROR RED),(FOLDING BUTTERFLY MIRROR HOT PINK),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf,1.0
1350,"(FOOD CONTAINER SET 3 LOVE HEART, CARD DOLLY G...",(10 COLOUR SPACEBOY PEN),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf,1.0
1355,(10 COLOUR SPACEBOY PEN),"(FOOD CONTAINER SET 3 LOVE HEART, CARD DOLLY G...",0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf,1.0
1380,"(CARD DOLLY GIRL, STRAWBERRY LUNCH BOX WITH CU...",(10 COLOUR SPACEBOY PEN),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf,1.0
